In [17]:
# !pip install foolbox==2.3.0

In [18]:
import foolbox
import numpy as np
import torch
import torchvision.models as models
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
from scipy.special import softmax
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import time 


In [19]:
# preprocess original image 
def preprocess_orig_image(image):

    im_orig = Image.open(image)
    im = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()])(im_orig)
    im = im.reshape(3,224,224)
    return im

# create foolbox model given the PyTorch model
def test_model(model):
    model = model.eval()
    preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
    fmodel = foolbox.models.PyTorchModel(model, bounds=(0, 1), num_classes=1000, preprocessing=preprocessing)
    return fmodel


def get_labels(label_orig, label_pert):

    labels = open(os.path.join('synset_words.txt'), 'r').read().split('\n')

    str_label_orig = labels[np.int(label_orig)].split(',')[0]
    str_label_pert = labels[np.int(label_pert)].split(',')[0]
    
    return str_label_orig,str_label_pert


# comptute the cosine similarity between 2 original image and adversarial image
def calculate_cossim(orig,pert):

    original_image = orig.flatten()
    perturbed_image = pert.flatten()

    dot_product = np.dot(original_image, perturbed_image)
    norm_a = np.linalg.norm(original_image)
    norm_b = np.linalg.norm(perturbed_image)
    cossimi = ((dot_product / (norm_a * norm_b)))
    return cossimi


def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])

    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err

In [15]:
model = models.inception_v3(pretrained=True) 
fmodel = test_model(model)
image = preprocess_orig_image("test_im1.jpg")
image = image.numpy()

image_label = np.argmax(fmodel.forward_one(image))
orig_confidence = np.amax(foolbox.utils.softmax(fmodel.forward_one(image)))

print ("Starting Attack")
start_time = time.time()
attack = foolbox.v1.attacks.CarliniWagnerL2Attack(fmodel)
adversarial = attack(image, image_label)
end_time = time.time() - start_time
print ("Attack Ended")

advs_label = np.argmax(fmodel.forward_one(adversarial))
advs_confidence = np.amax(foolbox.utils.softmax(fmodel.forward_one(adversarial)))

str_label_orig,str_label_advs = get_labels(image_label,advs_label)


print ("Actual Label:", str_label_orig, "(Confidence: %.2f%%) " % (orig_confidence*100))
print ("Predicted Label:", str_label_advs, "(Confidence: %.2f%%) " % (advs_confidence*100))
print ("Time Taken: %.2f" % end_time)


Starting Attack
Attack Ended
Actual Label: n01818515 macaw (Confidence: 100.00%) 
Predicted Label: n01817953 African grey (Confidence: 50.04%) 
Time Taken: 103.52


In [16]:
# print (adversarial)

orig = np.transpose(image, (1, 2, 0))
pert = np.transpose(adversarial, (1, 2, 0))
print (mse(orig,pert))
print (calculate_cossim(orig,pert))

1.0829351069079888e-05
0.9999932
